In [74]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *
import numpy as np

In [75]:
key_path = key_path
project = project_id

# Sources
table = 'bronze_ticker_info'
dataset = 'bronze'
table_conca = f'{project}.{dataset}.{table}'

# Loading tables
table_to_save = 'silver_ticker_info'
dataset_to_save = 'silver'
table_conca_to_save = f'{project}.{dataset_to_save}.{table_to_save}'


In [76]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [77]:
df = bigquery.run_query(
    f"""
    SELECT
        ticker,
        shortName,
        industry,
        sector,
        recommendationKey,
        fullTimeEmployees,
        marketCap,
        enterpriseValue,
        floatShares,
        sharesOutstanding,
        sharesShort,
        shortRatio,
        bookValue,
        currentPrice,
        totalCash,
        totalCashPerShare,
        totalDebt,
        totalRevenue,
        revenuePerShare,
        returnOnAssets,
        returnOnEquity,
        operatingCashflow,
        operatingMargins
    FROM {project}.{dataset}.{table}
    """
)
df

,ticker,shortName,industry,sector,recommendationKey,fullTimeEmployees,marketCap,enterpriseValue,floatShares,sharesOutstanding,...,currentPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue,revenuePerShare,returnOnAssets,returnOnEquity,operatingCashflow,operatingMargins
0,AGTI,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FSR,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GRPH,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GTXI,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ICVX,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10932,EVO,Evotec SE,Drug Manufacturers—Specialty & Generic,Healthcare,buy,5022.0,1.351537e+09,7.987532e+08,1.489987e+08,3.551070e+08,...,3.806,2.988660e+08,1.686,4.735110e+08,7.884410e+08,2.2265,0.00968,-0.15618,-5.490500e+07,-0.19850
10933,HLN,Haleon plc,Drug Manufacturers—Specialty & Generic,Healthcare,buy,25408.0,4.546196e+10,9.876281e+10,6.653338e+09,4.557590e+09,...,9.975,5.310000e+08,0.058,9.051000e+09,1.125800e+10,2.4460,0.04455,0.06822,2.193000e+09,0.23135
10934,CUBA,The Herzfeld Caribbean Basin Fu,Asset Management,Financial Services,none,NaN,3.420126e+07,NaN,NaN,1.513330e+07,...,2.260,NaN,NaN,NaN,7.391250e+05,0.0460,-0.00407,0.18404,NaN,-0.78019
10935,NYMTI,"New York Mortgage Trust, Inc. -",None,None,none,81.0,NaN,NaN,8.975719e+07,NaN,...,25.100,2.340820e+08,2.566,5.701235e+09,1.679370e+08,1.8460,-0.02527,-0.10293,2.822700e+07,-0.98756


In [78]:
tickers_indicators = bigquery.run_query(
    f"""
    SELECT DISTINCT
        ticker
    FROM {project}.silver.silver_indicators
    """
)
tickers_indicators

,ticker
0,PCAR
1,EQIX
2,CNC
3,ARE
4,EBAY
...,...
3440,ZEUS
3441,ELF
3442,MMSI
3443,SGBX


In [79]:
# Hacemos un INNER para traernos solo los ticker de los que tenemos historico

df_filtered = pd.merge(df, tickers_indicators, how='inner', on='ticker')
df_filtered = df_filtered[df.columns]
df_filtered

,ticker,shortName,industry,sector,recommendationKey,fullTimeEmployees,marketCap,enterpriseValue,floatShares,sharesOutstanding,...,currentPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue,revenuePerShare,returnOnAssets,returnOnEquity,operatingCashflow,operatingMargins
0,CDE,"Coeur Mining, Inc.",Gold,Basic Materials,buy,2074.0,2.487309e+09,3.093159e+09,392823043.0,399247008.0,...,6.230,74136000.0,0.186,6.360700e+08,8.917590e+08,2.3520,-0.00125,-0.07355,62272000.0,0.07511
1,CDE,"Coeur Mining, Inc.",Gold,Basic Materials,buy,2074.0,2.511264e+09,3.093159e+09,392823043.0,399247008.0,...,6.290,74136000.0,0.186,6.360700e+08,8.917590e+08,2.3520,-0.00125,-0.07355,62272000.0,0.07511
2,CDE,"Coeur Mining, Inc.",Gold,Basic Materials,buy,2074.0,2.524838e+09,3.093159e+09,392823043.0,399247008.0,...,6.324,74136000.0,0.186,6.360700e+08,8.917590e+08,2.3520,-0.00125,-0.07355,62272000.0,0.07511
3,CTGO,"Contango ORE, Inc.",Gold,Basic Materials,buy,6.0,2.596079e+08,3.125774e+08,8185519.0,12145400.0,...,21.375,24118918.0,1.986,7.459845e+07,NaN,NaN,NaN,NaN,-21586868.0,NaN
4,CTGO,"Contango ORE, Inc.",Gold,Basic Materials,buy,6.0,2.560250e+08,3.125774e+08,8185519.0,12145400.0,...,21.080,24118918.0,1.986,7.459845e+07,NaN,NaN,NaN,NaN,-21586868.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5751,CTLT,"Catalent, Inc.",Drug Manufacturers—Specialty & Generic,Healthcare,hold,18000.0,1.076469e+10,1.556915e+10,179854154.0,180980000.0,...,59.480,162000000.0,0.895,4.990000e+09,4.135000e+09,22.7510,-0.00712,-0.28290,250000000.0,0.00559
5752,HCM,HUTCHMED (China) Limited,Drug Manufacturers—Specialty & Generic,Healthcare,buy,1970.0,2.992986e+09,1.396775e+10,502034794.0,170888000.0,...,17.300,803510016.0,0.940,8.859400e+07,6.108060e+08,3.5750,-0.05902,-0.05439,-46977000.0,-0.09003
5753,AQST,"Aquestive Therapeutics, Inc.",Drug Manufacturers—Specialty & Generic,Healthcare,buy,135.0,4.006631e+08,3.510689e+08,79978698.0,91059800.0,...,4.400,89870000.0,0.987,3.572300e+07,5.836000e+07,0.7870,-0.10875,NaN,-24679000.0,0.00274
5754,EVO,Evotec SE,Drug Manufacturers—Specialty & Generic,Healthcare,buy,5022.0,1.351537e+09,7.987532e+08,148998744.0,355107008.0,...,3.806,298865984.0,1.686,4.735110e+08,7.884410e+08,2.2265,0.00968,-0.15618,-54905000.0,-0.19850


In [80]:
# Nos quedamos con solo una fila por ticker

# Reemplazar valores NaN en la columna 'market' con 0
df_filtered['marketCap'].fillna(0, inplace=True)

# Encontrar el índice de la fila con el valor más alto en 'market' para cada ticker
idx = df_filtered.groupby('ticker')['marketCap'].idxmax()

# Seleccionar esas filas del DataFrame original
df_filtered = df_filtered.loc[idx]

# Eliminamos los tickers que el marketcap sea 0
df_filtered = df_filtered[df_filtered['marketCap'] != 0]

df_filtered

,ticker,shortName,industry,sector,recommendationKey,fullTimeEmployees,marketCap,enterpriseValue,floatShares,sharesOutstanding,...,currentPrice,totalCash,totalCashPerShare,totalDebt,totalRevenue,revenuePerShare,returnOnAssets,returnOnEquity,operatingCashflow,operatingMargins
5466,A,"Agilent Technologies, Inc.",Diagnostics & Research,Healthcare,buy,18000.0,4.037208e+10,4.154804e+10,286385268.0,287529984.0,...,140.4100,1.779000e+09,6.187,2.932000e+09,6.497000e+09,22.269,0.08906,0.24658,1.786000e+09,0.21103
2208,AACG,ATA Creativity Global,Education & Training Services,Consumer Defensive,buy,581.0,1.855656e+07,3.715915e+07,18043830.0,31950000.0,...,0.5808,4.010650e+07,0.637,3.889986e+07,2.429485e+08,7.698,-0.05531,-0.35138,NaN,-0.40102
2884,AADI,"Aadi Bioscience, Inc.",Biotechnology,Healthcare,buy,70.0,4.110918e+07,-3.443604e+07,16148562.0,24614800.0,...,1.6701,7.856800e+07,3.192,1.056000e+06,2.381700e+07,0.883,-0.34932,-0.62264,-5.040500e+07,-2.52209
5011,AAL,"American Airlines Group, Inc.",Airlines,Industrials,hold,137400.0,6.583457e+09,3.716495e+10,647910648.0,656704000.0,...,10.0250,8.987000e+09,13.685,3.944700e+10,5.344800e+10,81.548,0.02779,NaN,2.015000e+09,0.09823
5308,AAME,Atlantic American Corporation,Insurance—Life,Financial Services,none,154.0,3.212969e+07,5.160161e+07,4055676.0,20399800.0,...,1.5750,2.121300e+07,1.040,4.012000e+07,1.860010e+08,9.117,-0.00146,-0.03087,2.615000e+06,0.00038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1939,ZUO,"Zuora, Inc.",Software—Infrastructure,Technology,buy,1618.0,1.343303e+09,1.225365e+09,139135713.0,139748992.0,...,9.0700,5.434710e+08,3.638,4.065180e+08,4.456830e+08,3.064,-0.02464,-0.31016,5.554000e+06,-0.07960
2923,ZVRA,"Zevra Therapeutics, Inc.",Biotechnology,Healthcare,buy,65.0,3.772959e+08,3.822178e+08,49294449.0,52617800.0,...,7.1705,4.925800e+07,0.936,5.946800e+07,2.368900e+07,0.603,-0.37412,-1.32319,-5.606000e+07,-5.34839
4013,ZWS,Zurn Elkay Water Solutions Corp,Pollution & Treatment Controls,Industrials,buy,2400.0,5.456876e+09,5.867595e+09,143434647.0,170928000.0,...,31.9250,1.627000e+08,0.952,5.452000e+08,1.541000e+09,8.908,0.05455,0.08043,3.007000e+08,0.17354
3291,ZYME,Zymeworks Inc.,Biotechnology,Healthcare,buy,290.0,8.177556e+08,5.098289e+08,39446221.0,71047400.0,...,11.5100,3.301460e+08,4.647,2.008800e+07,6.270500e+07,0.853,-0.13622,-0.26315,-9.137200e+07,-1.33030


In [81]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3100 entries, 5466 to 3400
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ticker             3100 non-null   object 
 1   shortName          3100 non-null   object 
 2   industry           3097 non-null   object 
 3   sector             3097 non-null   object 
 4   recommendationKey  3098 non-null   object 
 5   fullTimeEmployees  2953 non-null   float64
 6   marketCap          3100 non-null   float64
 7   enterpriseValue    3080 non-null   float64
 8   floatShares        3060 non-null   float64
 9   sharesOutstanding  3091 non-null   float64
 10  sharesShort        3096 non-null   float64
 11  shortRatio         3092 non-null   float64
 12  bookValue          3095 non-null   float64
 13  currentPrice       3098 non-null   float64
 14  totalCash          3081 non-null   float64
 15  totalCashPerShare  3075 non-null   float64
 16  totalDebt          3005 no

In [82]:
# Seleccionar solo las columnas numéricas
numerical_columns = df_filtered.select_dtypes(include=[np.number]).columns

# Identificar columnas numéricas con valores NaN
columns_with_nans = df_filtered[numerical_columns].columns[df_filtered[numerical_columns].isna().any()]

# Calcular la mediana para cada columna numérica con valores NaN
medianas = df_filtered[columns_with_nans].median()

# Rellenar los valores NaN en las columnas numéricas con la mediana correspondiente
df_filtered[columns_with_nans] = df_filtered[columns_with_nans].fillna(medianas)

df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3100 entries, 5466 to 3400
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ticker             3100 non-null   object 
 1   shortName          3100 non-null   object 
 2   industry           3097 non-null   object 
 3   sector             3097 non-null   object 
 4   recommendationKey  3098 non-null   object 
 5   fullTimeEmployees  3100 non-null   float64
 6   marketCap          3100 non-null   float64
 7   enterpriseValue    3100 non-null   float64
 8   floatShares        3100 non-null   float64
 9   sharesOutstanding  3100 non-null   float64
 10  sharesShort        3100 non-null   float64
 11  shortRatio         3100 non-null   float64
 12  bookValue          3100 non-null   float64
 13  currentPrice       3100 non-null   float64
 14  totalCash          3100 non-null   float64
 15  totalCashPerShare  3100 non-null   float64
 16  totalDebt          3100 no

In [83]:
# En el caso de tener un df en Bigquery, lo leemos y guardamos solo los nuevos registros
try:
    # Filtramos solamente los nuevos registros
    df_incremental = bigquery.select_for_incremental(id='ticker', table=table_conca, new_df=df)

    # Guardamos los datos en bigquery
    bigquery.save_dataframe(df_filtered, project, dataset, table, if_exists='replace', schema=None)

# En el caso de no tener datos en Bigquery, guardamos todo el df
except:
    bigquery.save_dataframe(df_filtered, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
